### <center>   Travail pratique 4 : Physique Numérique - PHY-3500

## <center> Équations différentielles partielles

#### <center>Par Julien Houle, Olivier Lapointe-Gagné et Pierre-Luc Larouche

***

Voici les différents modules, librairies et variables utilisées au travers du travail :

In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import e
import time
from scipy.constants import hbar
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.animation as animation
from IPython.display import HTML
Writer = animation.FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)

In [11]:
'''
Liste de constantes utiles
'''
m_e = 9.109e-31 # kg 

### Question 1

On peut d'abord implémenter la fonction d'onde de l'électron dans la boîte unidimensionnelle à $t=0$, puis l'utiliser pour produire un vecteur dont chaque composante corespond à la valeur de $\psi$ au départ à chaque incrément de position $a$, qui correspond ici à un millième du puit. Une troisième fonction construira alors les matrices tridiagonales $A$ ou $B$ d'un seul coup et une quatrième fonction en calculera le vecteur $v$ à partir de la matrice $B$ et du vecteur $\psi$.

In [12]:
def psi_0(x, L):
    '''
    Fonction qui calcule la fonction d'onde d'un électron au temps 0 à une position donné

    Paramètres: x: La position de l'électron, L: La longueur de la boîte

    Retourne: La valeur de psi 
    '''
    x_0 = L/2
    sig = 1e-10 #en mètres
    k = 5e10 # en 1/mètres
    psi = np.exp(-(x-x_0)**2/(2*sig**2))*np.exp(1j*k*x)
    return psi

In [13]:
def psi_0_vec(L,N):
    '''
    Fonction qui construit le vecteur psi(0) en fonction des pas de distance a

    Paramètres : L, la longueur de la boîte unidimensionnelle, N, le nombre de pas dans la boîte

    Retourne : le vecteur psi(0)
    '''
    a = L/N
    psi0 = np.empty([N+1,1],complex)
    for i in range(N+1):
        psi0[i]=psi_0(i*a,L)
    return psi0

In [14]:
def matrice(lettre,N,L,h):
    '''
    Fonction qui crée la matrice A ou B

    Paramètres: lettre: choix de matrice à créer, 
                N:nombre d'itérations positionnelles, L: longueur de la boîte, h:grandeur des itérations temporelles

    Retourne: une matrice tridiagonale qui constitue notre système d'équations différentielles
    '''
    a = L/N
    a_1 = 1 + h*1j*hbar/(2*m_e*a**2)
    a_2 = -h*1j*hbar/(4*m_e*a**2)
    b_1 = 1 - h*1j*hbar/(2*m_e*a**2)
    b_2 = h*1j*hbar/(4*m_e*a**2)
    matrice = np.zeros((N+1,N+1),complex)
    if lettre == 'A':
        for i in range(N+1):
            for l in range(N+1):
                if i == l:
                    matrice[i][l]  = a_1
                if i == l + 1 or i == l -1:
                    matrice[i][l] = a_2
    if lettre == 'B':
        for i in range(N+1):
            for l in range(N+1):
                if i == l:
                    matrice[i][l]  = b_1
                if i == l + 1 or i == l -1:
                    matrice[i][l] = b_2
    return matrice

In [15]:
def v_vec(L,N,h,psi):
    '''
    Fonction qui construit le vecteur v à partir de B et psi

    Paramètres: L:Longueur de la boîte, N: nombre de pas positionnel, h:grandeur des pas temporelles

    Retourne: le vecteur v recherché
    '''
    a = L/N
    b_1 = 1 - h*1j*hbar/(2*m_e*a**2)
    b_2 = h*1j*hbar/(4*m_e*a**2)
    v = np.empty((N+1,1),complex)
    v[0] = b_1*psi[0]+b_2*psi[1]
    v[N] = b_1*psi[N]+b_2*psi[N-1]
    for i in range(1,N):
        v[i] = b_1*psi[i]+b_2*(psi[i-1]+psi[i+1])
    return v

### Question 2

Une cinquième fonction, implémentant un algorithme de Thomas, permettra d'extraire le vecteur $x$, la valeur de la fonction d'onde pour chaque incrément de position au temps $h$ voulu, à partir de la matrice $A$ et du vecteur $v$.

In [16]:
def Thomas(MatriceIni, VecteurIni):
    '''
    Fonction qui utilise l'algo de Thomas pour résoudre AX = v

    Paramètres: Matrice est un matrice carré tridiagonale, Vecteur est une matrice vecteur

    Retourne: Un vecteur correspondant à X

    NOTE: Jesus saith unto him, Thomas, because thou hast seen me, thou hast believed:
    blessed are they that have not seen, and yet have believed
    '''
    taille = len(MatriceIni)
    Matrice = np.copy(MatriceIni)
    Vecteur = np.copy(VecteurIni)

    # Boucle qui fait la réduction de Gauss simplifiée sur la matrice et le vecteur.
    for i in range(taille - 1):
        Vecteur[i][0] /= (Matrice[i][i])
        Matrice[i] /= (Matrice[i][i])
        Vecteur[i + 1][0] -= (Matrice[i + 1][i]) * (Vecteur[i][0])
        Matrice[i + 1] -= ((Matrice[i + 1][i]) * (Matrice[i]))
    Vecteur[taille - 1][0] /= (Matrice[taille - 1][taille - 1])
    Matrice[taille - 1] /= (Matrice[taille - 1][taille - 1])

    # Boucle qui construit notre vecteur de sortie.
    for i in reversed(range(taille - 1)):
        Vecteur[i][0] -= (Matrice[i][i + 1]) * (Vecteur[i + 1][0])
    return Vecteur

### Question 3

On peut alors regrouper toutes ces fonctions dans une seule afin de calculer $\psi$ pour chaque $x$ à chaque incrément de temps $h$ selon la méthode de Crank-Nicolson et en ensuite en produire une animation.

In [17]:
def Crank_Nico(h,N,L,m):
    '''
    Fonction qui estime la valeur de psi en fonction du temps et de x avec la méthode de Crank-Nicolson

    Paramètres: h: grandeur des itérations temporelles, N: nombre d'itérations positionnelle, L:longueur de la boîte,
                m: nombre d'itérations temporelles

    Retourne: liste des états
    '''
    
    #On crée nos matrices
    A = matrice("A",N,1e-8,1e-18)
    
    #On crée notre vecteur initial
    psi = psi_0_vec(L,N)

    #On crée nos liste vides qui serviront à stocker nos points (eventuellement pour tracer)
    liste_x = [0]
    for i in range(N):
        liste_x.append(liste_x[-1]+(L/N))
    liste_psi = []
    liste_etats = []

    #On crée un compteur pour le temps
    t=0
    
    #On crée le premier état (t=0)
    etat_1=np.transpose(psi)[0]
    liste_psi = etat_1
    liste_etats.append(liste_psi)

    #On crée une boucle infini
    while t<m*1e-18:
        #On augmente notre compteur de temps de h
        t += h
        #On applique la méhode de thomas pour trouver le deuxième etat
        v= v_vec(L,N,h,psi)
        psi = Thomas(A,v)

        etat = np.transpose(psi)[0]
        liste_psi = etat

        liste_etats.append(liste_psi)
    return liste_etats

In [18]:
'''
Programmons l'animation da fonction d'onde
'''

liste_y = Crank_Nico(1e-18,1000,1e-8,1000)
liste_x = [0]
for i in range(1000):
    liste_x.append(liste_x[-1]+((1e-8)/1000))

fig = plt.figure()


def animate(i):
    x = liste_x
    y = np.real(liste_y[i]) 
    fig.clear() 
    plt.plot(x,y)
    plt.plot(x,y)
    plt.ylim(-1,1)

ani = animation.FuncAnimation(fig, animate, interval=1,blit=False)
plt.show()
plt.close(ani._fig)
HTML(ani.to_html5_video())

<Figure size 432x288 with 0 Axes>

RuntimeError: Requested MovieWriter (ffmpeg) not available

In [ ]:
#Crank_Nico(1e-18,1000,1e-8)

### Question 4

Idem qu'à la question précédante, en remplaçant seulement le calcul d'algèbre linéaire exécuté par l'algorithme de Thomas pas celui déjà implémenté dans Numpy, soit linalg. On constate le même résultat numérique pour les deux, seulement un grand gain en temps en utilisant notre propre méthode de calcul. 

In [ ]:
def Crank_Nico_linalg(h,N,L):
    '''
    Fonction qui estime la valeur de psi en fonction du temps et de x avec la méthode de Crank-Nicolson

    Paramètres: h: grandeur des itérations temporelles, N: nombre d'itérations positionnelle, L:longueur de la boîte

    Retourne:
    '''

    #On crée une figure pyplot
    plt.ion()

    figure, ax = plt.subplots(figsize=(8, 6))
    
    #On crée nos matrices
    A = matrice("A",N,1e-8,1e-18)
    
    #On crée notre vecteur initial
    psi = psi_0_vec(L,N)

    #On crée nos liste vides qui serviront à stocker nos points (eventuellement pour tracer)
    liste_x = np.arange(0,L,L/(N+1))
    liste_z =[]
    for i in range(len(liste_x)):
        liste_z.append(0)
    liste_psi = []

    #On crée un compteur pour le temps
    t=0
    
    #On crée le premier état (t=0)
    etat_1=np.transpose(np.real(psi))
    etat_2 = np.transpose(np.imag(psi))
    liste_psi = etat_1
    liste_zim = etat_2

    #On plot le premier etat
    ax = plt.axes(projection='3d')
    line = ax.plot3D(liste_x, liste_psi[0],  liste_zim[0], c='blue')
    plt.show()
    plt.pause(0.2)
    #On crée une boucle infini
    while True:
        #On augmente notre compteur de temps de h
        t += h
        #On applique la méhode de thomas pour trouver le deuxième etat
        v= v_vec(L,N,h,psi)
        psi = np.linalg.solve(A,v)
        etat = np.transpose(np.real(psi))
        etat2 = np.transpose(np.imag(psi))
        liste_psi = etat
        liste_zim = etat2
        #On plot le premier etat
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1,1)
        line = ax.plot3D(liste_x, liste_psi[0],  liste_zim[0], c='blue')
        figure.show()
        figure.canvas.flush_events()
        time.sleep(0.00001)
        ax.cla()

In [ ]:
Crank_Nico_linalg(1e-18,1000,1e-8)

### Question 5

La fonction d'onde se déplace au fil du temps de sa position au temps $t=0$ jusqu'à l'extrémité droite de la boîte unidimensionnelle lors des premiers incréments de temps. Ce comportement est attendu comme la fonction initiale implique une exponentielle complexe positive. En s'approchant de la parois, la fonction d'onde oscille de plus en plus énergétiquement, symbole que sa densité de probablité est de plus en plus incertaine, sa probabilité de présence étant de plus en plus basse en s'y approchant. On peut reconnaître ensuite une sorte de réflection dure de la fonction d'onde sur la parois, qui retournera à sa position intiale avant se déplacer vers la seconde parois de la boîte. Comme les comportements de la fonction d'onde calculé par notre fonction implémentée et celle de numpy.linalg sont identiques, on peut en déduire que notre algorithme est un moyen efficace de rapidement résoudre numériquement l'équation de Shroedinger pour cette particule dans une boîte.